In [1]:
import emmental
from emmental.modules.identity_module import IdentityModule
from emmental.task import Task
from emmental.emmental_model import EmmentalModel
from torch import nn
import torch

In [2]:
def print_params(task):
    for key in task.module_pool.keys():
        print(key)
        for param in task.module_pool[key].named_parameters():
            print(param)

In [3]:
def print_params_all(task):
    for key in task.module_pool.keys():
        print(key)
        for param in task.module_pool[key].named_parameters():
            print(param)

In [4]:
task1 = Task(
    name="linear1",
    module_pool=nn.ModuleDict(
        {
            "im1": IdentityModule(),
            "linear1": nn.Linear(4, 2, bias=False),
            "linear2": nn.Linear(2, 1, bias=False),
        }
    ),
    task_flow=[
        {"module": "linear1", "inputs": [(0, 0)]},
        {"module": "linear2", "inputs": [(1, 0)]},
    ],
    loss_func=None,
    output_func=None,
)

In [5]:
task2 = Task(
    name="linear2",
    module_pool=nn.ModuleDict(
        {"im2": IdentityModule(), "linear1": nn.Linear(4, 2, bias=False)}
    ),
    task_flow=None,
    loss_func=None,
    output_func=None,
)

In [6]:
print_params(task1)
print_params(task2)

im1
linear1
('weight', Parameter containing:
tensor([[-0.4613, -0.4269,  0.4455,  0.0602],
        [-0.2865,  0.4622, -0.3994, -0.2003]], requires_grad=True))
linear2
('weight', Parameter containing:
tensor([[ 0.2056, -0.5968]], requires_grad=True))
im2
linear1
('weight', Parameter containing:
tensor([[ 0.0831,  0.1457,  0.1203, -0.2610],
        [ 0.4883, -0.0560,  0.0956, -0.0648]], requires_grad=True))


In [7]:
mtl_model = EmmentalModel(name = 'all', tasks=[task1, task2])

In [8]:
mtl_model

EmmentalModel(name=all)

In [9]:
print_params(task1)
print('====')
print_params(task2)
print('====')
print_params_all(mtl_model)

im1
linear1
('weight', Parameter containing:
tensor([[-0.4613, -0.4269,  0.4455,  0.0602],
        [-0.2865,  0.4622, -0.3994, -0.2003]], requires_grad=True))
linear2
('weight', Parameter containing:
tensor([[ 0.2056, -0.5968]], requires_grad=True))
====
im2
linear1
('weight', Parameter containing:
tensor([[-0.4613, -0.4269,  0.4455,  0.0602],
        [-0.2865,  0.4622, -0.3994, -0.2003]], requires_grad=True))
====
im1
linear1
('weight', Parameter containing:
tensor([[-0.4613, -0.4269,  0.4455,  0.0602],
        [-0.2865,  0.4622, -0.3994, -0.2003]], requires_grad=True))
linear2
('weight', Parameter containing:
tensor([[ 0.2056, -0.5968]], requires_grad=True))
im2


In [10]:
type(task1.module_pool)

torch.nn.modules.container.ModuleDict

In [11]:
type(task2.module_pool)

torch.nn.modules.container.ModuleDict

In [12]:
type(mtl_model.module_pool)

torch.nn.modules.container.ModuleDict

In [13]:
task_flow = [
    {"module": "linear1", "inputs": [(0, 0)]},
    {"module": "linear2", "inputs": [(1, 0)]},
]

In [14]:
var1 = torch.tensor([1.,2.,3.,4.])

In [15]:
mtl_model.module_pool['linear2'].forward(mtl_model.module_pool['linear1'].forward(var1))

tensor([0.8664], grad_fn=<SqueezeBackward3>)

In [16]:
import numpy as np

In [17]:
m1 = np.array([[-0.1828, -0.1443, -0.2745, -0.4306], [0.4322, -0.0427, 0.1007, 0.4297]])
m1.shape

(2, 4)

In [18]:
m2 = np.array([0.5048, 0.4704])
m2.shape

(2,)

In [19]:
v1 = np.array([1.0, 2.0, 3.0, 4.0]).reshape((1,4))

In [20]:
v1 @ m1.transpose() @ m2.transpose()

array([-0.40936696])

In [21]:
mtl_model.forward(var1, task_names=['linear1'])

{'linear1': [[tensor([1., 2., 3., 4.])],
  [tensor([ 0.2624, -1.3615], grad_fn=<SqueezeBackward3>)],
  [tensor([0.8664], grad_fn=<SqueezeBackward3>)]]}

In [22]:
immediate_ouput = [[var1]]
for action in task_flow:
    print(action)
    print('=========')
    input = [immediate_ouput[action_index][output_index] for action_index, output_index in action['inputs']]
    print('input', input)
    output = mtl_model.module_pool[action['module']].forward(*input)
    print('output', output)
    if isinstance(output, tuple): output = list(output)
    if not isinstance(output, list): output = [output]
    immediate_ouput.append(output)
    print(immediate_ouput)
task_flow

{'module': 'linear1', 'inputs': [(0, 0)]}
input [tensor([1., 2., 3., 4.])]
output tensor([ 0.2624, -1.3615], grad_fn=<SqueezeBackward3>)
[[tensor([1., 2., 3., 4.])], [tensor([ 0.2624, -1.3615], grad_fn=<SqueezeBackward3>)]]
{'module': 'linear2', 'inputs': [(1, 0)]}
input [tensor([ 0.2624, -1.3615], grad_fn=<SqueezeBackward3>)]
output tensor([0.8664], grad_fn=<SqueezeBackward3>)
[[tensor([1., 2., 3., 4.])], [tensor([ 0.2624, -1.3615], grad_fn=<SqueezeBackward3>)], [tensor([0.8664], grad_fn=<SqueezeBackward3>)]]


[{'module': 'linear1', 'inputs': [(0, 0)]},
 {'module': 'linear2', 'inputs': [(1, 0)]}]

In [23]:
x = [
    torch.Tensor([1]),
    torch.Tensor([1, 2]),
    torch.Tensor([1, 2, 3]),
    torch.Tensor([1, 2, 3, 4]),
    torch.Tensor([1, 2, 3, 4, 5]),
]

In [24]:
y1 = [
    torch.Tensor([0]),
    torch.Tensor([0]),
    torch.Tensor([1]),
    torch.Tensor([1]),
    torch.Tensor([1]),
]

y2 = [
    torch.Tensor([1]),
    torch.Tensor([1]),
    torch.Tensor([0]),
    torch.Tensor([0]),
    torch.Tensor([0]),
]

In [25]:
from emmental.data import EmmentalDataset, EmmentalDataLoader

In [26]:
dataset = EmmentalDataset({"data": x}, {"label1": y1, "label2": y2})

In [27]:
dataset[4]

({'data': tensor([1., 2., 3., 4., 5.])},
 {'label1': tensor([1.]), 'label2': tensor([0.])})

In [28]:
dataloader1 = EmmentalDataLoader(dataset, batch_size=2)

In [29]:
for x_batch, y_batch in dataloader1:
    print(x_batch, y_batch)
    for key, value in x_batch.items():
        print(key, value.size())
    for key, value in y_batch.items():
        print(key, value.size())

{'data': tensor([[1., 0.],
        [1., 2.]])} {'label1': tensor([[0.],
        [0.]]), 'label2': tensor([[1.],
        [1.]])}
data torch.Size([2, 2])
label1 torch.Size([2, 1])
label2 torch.Size([2, 1])
{'data': tensor([[1., 2., 3., 0.],
        [1., 2., 3., 4.]])} {'label1': tensor([[1.],
        [1.]]), 'label2': tensor([[0.],
        [0.]])}
data torch.Size([2, 4])
label1 torch.Size([2, 1])
label2 torch.Size([2, 1])
{'data': tensor([[1., 2., 3., 4., 5.]])} {'label1': tensor([[1.]]), 'label2': tensor([[0.]])}
data torch.Size([1, 5])
label1 torch.Size([1, 1])
label2 torch.Size([1, 1])


In [30]:
dataloader2 = EmmentalDataLoader(dataset, batch_size=3)

In [31]:
for x_batch, y_batch in dataloader2:
    print(x_batch, y_batch)
    for key, value in x_batch.items():
        print(key, value.size())
    for key, value in y_batch.items():
        print(key, value.size())

{'data': tensor([[1., 0., 0.],
        [1., 2., 0.],
        [1., 2., 3.]])} {'label1': tensor([[0.],
        [0.],
        [1.]]), 'label2': tensor([[1.],
        [1.],
        [0.]])}
data torch.Size([3, 3])
label1 torch.Size([3, 1])
label2 torch.Size([3, 1])
{'data': tensor([[1., 2., 3., 4., 0.],
        [1., 2., 3., 4., 5.]])} {'label1': tensor([[1.],
        [1.]]), 'label2': tensor([[0.],
        [0.]])}
data torch.Size([2, 5])
label1 torch.Size([2, 1])
label2 torch.Size([2, 1])


In [32]:
dataset.Y_dict['label1'][0] = torch.Tensor([2.])

In [33]:
for x_batch, y_batch in dataloader1:
    print(x_batch, y_batch)
    for key, value in x_batch.items():
        print(key, value.size())
    for key, value in y_batch.items():
        print(key, value.size())

{'data': tensor([[1., 0.],
        [1., 2.]])} {'label1': tensor([[2.],
        [0.]]), 'label2': tensor([[1.],
        [1.]])}
data torch.Size([2, 2])
label1 torch.Size([2, 1])
label2 torch.Size([2, 1])
{'data': tensor([[1., 2., 3., 0.],
        [1., 2., 3., 4.]])} {'label1': tensor([[1.],
        [1.]]), 'label2': tensor([[0.],
        [0.]])}
data torch.Size([2, 4])
label1 torch.Size([2, 1])
label2 torch.Size([2, 1])
{'data': tensor([[1., 2., 3., 4., 5.]])} {'label1': tensor([[1.]]), 'label2': tensor([[0.]])}
data torch.Size([1, 5])
label1 torch.Size([1, 1])
label2 torch.Size([1, 1])


In [34]:
for x_batch, y_batch in dataloader2:
    print(x_batch, y_batch)
    for key, value in x_batch.items():
        print(key, value.size())
    for key, value in y_batch.items():
        print(key, value.size())

{'data': tensor([[1., 0., 0.],
        [1., 2., 0.],
        [1., 2., 3.]])} {'label1': tensor([[2.],
        [0.],
        [1.]]), 'label2': tensor([[1.],
        [1.],
        [0.]])}
data torch.Size([3, 3])
label1 torch.Size([3, 1])
label2 torch.Size([3, 1])
{'data': tensor([[1., 2., 3., 4., 0.],
        [1., 2., 3., 4., 5.]])} {'label1': tensor([[1.],
        [1.]]), 'label2': tensor([[0.],
        [0.]])}
data torch.Size([2, 5])
label1 torch.Size([2, 1])
label2 torch.Size([2, 1])


In [35]:
dataset[0]

({'data': tensor([1.])}, {'label1': tensor([2.]), 'label2': tensor([1.])})